In [1]:
from pyesgf.search import SearchConnection
import os
import pandas as pd
import requests
from tqdm import tqdm
os.environ["ESGF_PYCLIENT_NO_FACETS_STAR_WARNING"] = "on"

c:\Users\USER2\anaconda3\Lib\site-packages\pandas\core\arrays\masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


In [2]:
conn = SearchConnection('https://esgf-node.llnl.gov/esg-search', distrib=True)

In [3]:
query = conn.new_context(
    latest = True,
    project='CMIP6',
    source_id= 'EC-Earth3',
    experiment_id='ssp126',
    variable_id= 'hur',
    table_id="day",
    member_id='r1i1p1f1',
    data_node='esgf-data1.llnl.gov')

In [4]:
results = query.search()

In [5]:
#glitchy work-around
for i in range(0,2):
    try:
        hit = results[0].file_context().search()
        print(i)
    except:
        continue

0
1
2


In [15]:
files = map(lambda f : {'filename': f.filename, 'url': f.download_url}, hit)
files

In [16]:
files = list(files)

In [17]:
for i in range(2, len(results)):
    files.extend(list(map(lambda f : {'filename': f.filename, 'url': f.download_url},
                               results[i].file_context().search())))

In [18]:
files = pd.DataFrame.from_dict(files)
files

,filename,url
0,hur_day_EC-Earth3_ssp126_r1i1p1f1_gr_20150101-...,https://esgf-data1.llnl.gov/thredds/fileServer...
1,hur_day_EC-Earth3_ssp126_r1i1p1f1_gr_20160101-...,https://esgf-data1.llnl.gov/thredds/fileServer...
2,hur_day_EC-Earth3_ssp126_r1i1p1f1_gr_20170101-...,https://esgf-data1.llnl.gov/thredds/fileServer...
3,hur_day_EC-Earth3_ssp126_r1i1p1f1_gr_20180101-...,https://esgf-data1.llnl.gov/thredds/fileServer...
4,hur_day_EC-Earth3_ssp126_r1i1p1f1_gr_20190101-...,https://esgf-data1.llnl.gov/thredds/fileServer...
...,...,...
81,hur_day_EC-Earth3_ssp126_r1i1p1f1_gr_20960101-...,https://esgf-data1.llnl.gov/thredds/fileServer...
82,hur_day_EC-Earth3_ssp126_r1i1p1f1_gr_20970101-...,https://esgf-data1.llnl.gov/thredds/fileServer...
83,hur_day_EC-Earth3_ssp126_r1i1p1f1_gr_20980101-...,https://esgf-data1.llnl.gov/thredds/fileServer...
84,hur_day_EC-Earth3_ssp126_r1i1p1f1_gr_20990101-...,https://esgf-data1.llnl.gov/thredds/fileServer...


In [19]:
def download(url, filename):
    print("Downloading ", filename)
    r = requests.get(url, stream=True)
    total_size, block_size = int(r.headers.get('content-length', 0)), 1024
    with open(filename, 'wb') as f:
        for data in tqdm(r.iter_content(block_size),
                         total=total_size//block_size,
                         unit='KiB', unit_scale=True):
            f.write(data)

    if total_size != 0 and os.path.getsize(filename) != total_size:
        print("Downloaded size does not match expected size!\n",
              "FYI, the status code was ", r.status_code)

In [14]:
for i in range(67, 76,1):
    download(files.url[i],files.filename[i])

1.31MKiB [14:32, 1.50kKiB/s]                          


1.31MKiB [14:49, 1.47kKiB/s]                          


1.31MKiB [14:52, 1.47kKiB/s]                          


1.31MKiB [15:17, 1.43kKiB/s]                          


1.31MKiB [15:11, 1.44kKiB/s]                          


1.31MKiB [15:32, 1.40kKiB/s]                          


1.31MKiB [15:21, 1.42kKiB/s]                          


1.31MKiB [16:04, 1.36kKiB/s]                          


1.31MKiB [15:43, 1.39kKiB/s]                          
